In [2]:
import numpy as np 
import pandas as pd
import os
import os.path as path
print(os.listdir("../input"))
import glob
import cv2
import pickle
from keras.utils import to_categorical
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, Activation,Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import backend as k
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

FIG_WIDTH=20 # width of figure
HEIGHT_PER_ROW=3 # height of each row when showing a figure which consists of multiple rows
RESIZE_DIM=32 # images will be resized to 32x32 pixels
MODEL_NAME = 'bangla_digit_convnet_new'

def load_data():
    data_dir=os.path.join('..','input')
    paths_train_a=glob.glob(os.path.join(data_dir,'training-a','*.png'))
    paths_train_b=glob.glob(os.path.join(data_dir,'training-b','*.png'))
    paths_train_e=glob.glob(os.path.join(data_dir,'training-e','*.png'))
    paths_train_c=glob.glob(os.path.join(data_dir,'training-c','*.png'))
    paths_train_d=glob.glob(os.path.join(data_dir,'training-d','*.png'))
    paths_train_all=paths_train_a+paths_train_b+paths_train_c+paths_train_d+paths_train_e

    paths_test_a=glob.glob(os.path.join(data_dir,'testing-a','*.png'))
    paths_test_b=glob.glob(os.path.join(data_dir,'testing-b','*.png'))
    paths_test_e=glob.glob(os.path.join(data_dir,'testing-e','*.png'))
    paths_test_c=glob.glob(os.path.join(data_dir,'testing-c','*.png'))
    paths_test_d=glob.glob(os.path.join(data_dir,'testing-d','*.png'))
    paths_test_f=glob.glob(os.path.join(data_dir,'testing-f','*.png'))+glob.glob(os.path.join(data_dir,'testing-f','*.JPG'))
    paths_test_auga=glob.glob(os.path.join(data_dir,'testing-auga','*.png'))
    paths_test_augc=glob.glob(os.path.join(data_dir,'testing-augc','*.png'))
    paths_test_all=paths_test_a+paths_test_b+paths_test_c+paths_test_d+paths_test_e+paths_test_f+paths_test_auga+paths_test_augc
    
    path_label_train_a=os.path.join(data_dir,'training-a.csv')
    path_label_train_b=os.path.join(data_dir,'training-b.csv')
    path_label_train_e=os.path.join(data_dir,'training-e.csv')
    path_label_train_c=os.path.join(data_dir,'training-c.csv')
    path_label_train_d=os.path.join(data_dir,'training-d.csv')
    
    X_train_a,y_train_a=get_data(paths_train_a,path_label_train_a,resize_dim=RESIZE_DIM)
    X_train_b,y_train_b=get_data(paths_train_b,path_label_train_b,resize_dim=RESIZE_DIM)
    X_train_c,y_train_c=get_data(paths_train_c,path_label_train_c,resize_dim=RESIZE_DIM)
    X_train_d,y_train_d=get_data(paths_train_d,path_label_train_d,resize_dim=RESIZE_DIM)
    X_train_e,y_train_e=get_data(paths_train_e,path_label_train_e,resize_dim=RESIZE_DIM)
    
    X_train_all=np.concatenate((X_train_a,X_train_b,X_train_c,X_train_d,X_train_e),axis=0)
    y_train_all=np.concatenate((y_train_a,y_train_b,y_train_c,y_train_d,y_train_e),axis=0)
    
    X_show_all=X_train_all
    
    X_test_a=get_data(paths_test_a,resize_dim=RESIZE_DIM)
    X_test_b=get_data(paths_test_b,resize_dim=RESIZE_DIM)
    X_test_c=get_data(paths_test_c,resize_dim=RESIZE_DIM)
    X_test_d=get_data(paths_test_d,resize_dim=RESIZE_DIM)
    X_test_e=get_data(paths_test_e,resize_dim=RESIZE_DIM)
    X_test_f=get_data(paths_test_f,resize_dim=RESIZE_DIM)
    X_test_auga=get_data(paths_test_auga,resize_dim=RESIZE_DIM)
    X_test_augc=get_data(paths_test_augc,resize_dim=RESIZE_DIM)
    
    X_test_all=np.concatenate((X_test_a,X_test_b,X_test_c,X_test_d,X_test_e,X_test_f,X_test_auga,X_test_augc))
    
    X_tshow_all=X_test_all
    
    X_train_all = X_train_all.reshape(X_train_all.shape[0],32, 32,1).astype('float32')
    X_test_all = X_test_all.reshape(X_test_all.shape[0],32, 32,1).astype('float32')
    
    X_train_all = X_train_all/255
    X_test_all=X_test_all/255
    
    indices=list(range(len(X_train_all)))
    np.random.seed(42)
    np.random.shuffle(indices)
    
    ind=int(len(indices)*0.80)
    # train data
    X_train=X_train_all[indices[:ind]] 
    y_train=y_train_all[indices[:ind]]
    # validation data
    X_val=X_train_all[indices[-(len(indices)-ind):]] 
    y_val=y_train_all[indices[-(len(indices)-ind):]]
    
    return X_train, y_train, X_val, y_val
    
def get_key(path):
    # seperates the key of an image from the filepath
    key=path.split(sep=os.sep)[-1]
    return key
    
def get_data(paths_img,path_label=None,resize_dim=None):
    X=[] # initialize empty list for resized images
    for i,path in enumerate(paths_img):
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE) # images loaded in color (BGR)
        if resize_dim is not None:
            img=cv2.resize(img,(resize_dim,resize_dim),interpolation=cv2.INTER_AREA
        
        gaussian = cv2.GaussianBlur(img, (9,9), 10.0) #unblur
        img = cv2.addWeighted(img, 1.5, gaussian, -0.5, 0, img)
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
        img = cv2.filter2D(img, -1, kernel)
        ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        X.append(img)
        
    X=np.array(X) # tranform list to numpy array
    if  path_label is None:
        return X
    else:
        df = pd.read_csv(path_label) # read labels
        df=df.set_index('filename') 
        y_label=[df.loc[get_key(path)]['digit'] for path in  paths_img] # get the labels corresponding to the images
        y=to_categorical(y_label,10) # transfrom integer value to categorical variable
        return X, y
    
def my_model(img_size,channels):
    model = Sequential()
    input_shape = (img_size,img_size,channels)
    model.add(Conv2D(32, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    
    return model


def train_model(X_train,y_train,X_val,y_val):
    path_model='model_filter.h5' # location of saving model after each epoch
    K.tensorflow_backend.clear_session() # destroys the current graph and builds a new one
    model=my_model(32,1) # create the model
    K.set_value(model.optimizer.lr,1e-3) # set the learning rate
    # fit the model
    h=model.fit(x=X_train,     
                y=y_train, 
                batch_size=64, 
                epochs=30, 
                verbose=1, 
                validation_data=(X_val,y_val),
                shuffle=True,
                callbacks=[
                    ModelCheckpoint(filepath=path_model),
                ]
                )
                
def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')
    
    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")
    

def main():
    if not path.exists('out'):
        os.mkdir('out')

    X_train, y_train, X_val, y_val = load_data()

    train_model(X_train,y_train,X_val,y_val)
    
    model = my_model(32,1)

    export_model(tf.train.Saver(), model, ["conv2d_1_input"], "activation_2/Softmax")
    
if __name__ == '__main__':
    main()    
                
 

SyntaxError: invalid syntax (<ipython-input-2-ead3e842d5b7>, line 109)